In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 5860, done.
remote: Counting objects: 100% (1049/1049), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 5860 (delta 879), reused 834 (delta 706), pack-reused 4811
Receiving objects: 100% (5860/5860), 2.94 GiB | 31.58 MiB/s, done.
Resolving deltas: 100% (4705/4705), done.
Updating files: 100% (496/496), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 6.9 MB/s eta 0:00:00


In [3]:
# ! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_SVDpp_Cython, MatrixFactorization_AsySVD_Cython, MatrixFactorization_WARP_Cython
from Recommenders.MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping

from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.HHPRecommender import HHPRecommender

from Recommenders.BaseRecommender import BaseRecommender

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_twra.db',
    'copy_prev_best_params': False,
    'save_github': True
}

In [10]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_twra.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [11]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [12]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [13]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

https://arxiv.org/pdf/2004.10393.pdf

In [14]:
class TwoWayRankAggregationRecommender(BaseRecommender):
    '''
    TWRA (Two Way Rank Aggregation) is an algorithm designed to work with graph algorithms that computes
    score of user u and item i as a convex combination of type:
    
    score(u, i) = (1 - lambda) * f-score(u, i) + lambda * b-score(u, i)
    
    where f-score is obtained by apply a graph based algorithm (ProbS, HeatS, P3alpha, RP3beta, HHP, ...)
    in the usual way and b-score is obtained by applying the same (or another) algorithm but with the role 
    of users and items reversed. For example, with ProbS (P3), the 3 step random walk starts from an item 
    and not from a user.
    
    In principle, this idea can be extended also to algorithm that do not work with graphs.
    
    Original Paper: https://arxiv.org/pdf/2004.10393.pdf
    '''
    
    RECOMMENDER_NAME = "TwoWayRankAggregationRecommender"
    AVAILABLE_ALGORITHMS = ['p3', 'p3alpha', 'rp3beta', 'hhp']
    
    def __init__(self, URM_train):
        super(TwoWayRankAggregationRecommender, self).__init__(URM_train)
        
        self.bURM_train = self.URM_train.transpose(copy=True).tocsr()
        
    def fit(
        self, 
        base_f_algorithm: str = 'p3', 
        base_b_algorithm: str = 'p3', 
        convex_lambda: float = 0.5, # must be between 0 and 1
        f_params: dict = None, 
        b_params: dict = None
    ):
        
        if base_f_algorithm not in self.AVAILABLE_ALGORITHMS:
            raise ValueError("Value for 'base_f_algorithm' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_ALGORITHMS, base_f_algorithm))
            
        if base_b_algorithm not in self.AVAILABLE_ALGORITHMS:
            raise ValueError("Value for 'base_b_algorithm' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_ALGORITHMS, base_b_algorithm))
            
        if convex_lambda < 0.0 or convex_lambda > 1.0:
            raise ValueError("Value for 'convex_lambda' not valid. Must be inside [0, 1]")
        
        self.base_f_algorithm = base_f_algorithm
        self.base_b_algorithm = base_b_algorithm
        self.convex_lambda = convex_lambda
        self.f_params = f_params if f_params is not None else {}
        self.b_params = b_params if b_params is not None else {}

        if self.base_f_algorithm == 'p3':
            if 'alpha' in self.f_params:
                self.f_params['alpha'] = 1.0
            self.f_recommender = P3alphaRecommender(self.URM_train)
        elif self.base_f_algorithm == 'p3alpha':
            self.f_recommender = P3alphaRecommender(self.URM_train)
        elif self.base_f_algorithm == 'rp3beta':
            self.f_recommender = RP3betaRecommender(self.URM_train)
        else:
            self.f_recommender = HHPRecommender(self.URM_train)
            
        if self.base_b_algorithm == 'p3':
            if 'alpha' in self.b_params:
                self.b_params['alpha'] = 1.0
            self.b_recommender = P3alphaRecommender(self.bURM_train)
        elif self.base_b_algorithm == 'p3alpha':
            self.b_recommender = P3alphaRecommender(self.bURM_train)
        elif self.base_b_algorithm == 'rp3beta':
            self.b_recommender = RP3betaRecommender(self.bURM_train)
        else:
            self.b_recommender = HHPRecommender(self.bURM_train)
            
        self.f_recommender.fit(**self.f_params)
        self.b_recommender.fit(**self.b_params)
        
    def _compute_item_score(self, user_id_array, items_to_compute=None):
        
        f_scores = self.f_recommender._compute_item_score(user_id_array, items_to_compute)
        
        if items_to_compute:
            b_scores = -np.inf * np.ones((self.n_items, len(user_id_array)))
            b_scores[items_to_compute, :] = self.b_recommender._compute_item_score(items_to_compute, items_to_compute=user_id_array)[:, user_id_array]
        else:
            b_scores = self.b_recommender._compute_item_score(np.arange(self.n_items), items_to_compute=user_id_array)[:, user_id_array]
        
        return (1 - self.convex_lambda) * f_scores + self.convex_lambda * b_scores.T

In [15]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10]) 

recommender = TwoWayRankAggregationRecommender(URM_train)
recommender.fit()
result_df, _ = evaluator.evaluateRecommender(recommender)
result_df

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
TwoWayRankAggregationRecommender: URM Detected 627 ( 4.8%) users with no interactions.
TwoWayRankAggregationRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) users with no interactions.
P3alphaRecommender: URM Detected 627 ( 4.8%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2080.06 column/sec. Elapsed time 10.74 sec
P3alphaRecommender: Similarity column 13024 (100.0%), 2234.14 column/sec. Elapsed time 5.83 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 55.53 sec. Users per second: 188


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.038437,0.065467,0.054482,0.017245,0.030039,0.113719,0.058606,0.045075,0.259468,0.138378,...,0.802826,0.208308,0.802826,0.17669,12.028736,0.999566,0.509822,0.926647,0.990597,0.337237


In [16]:
def objective_function(trial):
    params = {
        'base_f_algorithm': trial.suggest_categorical('base_f_algorithm', ['p3', 'p3alpha', 'rp3beta', 'hhp']), 
        'base_b_algorithm': trial.suggest_categorical('base_b_algorithm', ['p3', 'p3alpha', 'rp3beta', 'hhp']), 
        'convex_lambda': trial.suggest_float('convex_lambda', 0.01, 0.99),
    }
    
    if params['base_f_algorithm'] == 'p3':
        params['f_params'] = {
            'topK': trial.suggest_int('f_p3_topK', 5, 2000, log=True),
            'implicit': True, 
            'normalize_similarity': True
        }
    elif params['base_f_algorithm'] == 'p3alpha':
        params['f_params'] = {
            'alpha': trial.suggest_float('f_p3a_alpha', 1e-5, 100, log=True),
            'topK': trial.suggest_int('f_p3a_topK', 5, 2000, log=True),
            'implicit': True, 
            'normalize_similarity': True
        }
    elif params['base_f_algorithm'] == 'rp3beta':
        params['f_params'] = {
            'alpha': trial.suggest_float('f_rp3_alpha', 1e-5, 100, log=True),
            'beta': trial.suggest_float('f_rp3_beta', 1e-5, 100, log=True),
            'topK': trial.suggest_int('f_rp3_topK', 5, 2000, log=True),
            'implicit': True, 
            'normalize_similarity': True
        }
    else:
        params['f_params'] = {
            'hybrid_lambda': trial.suggest_float('f_hhp_hybrid_lambda', 0.01, 0.99),
            'topK': trial.suggest_int('f_hhp_topK', 5, 2000, log=True),
        }
        
    if params['base_b_algorithm'] == 'p3':
        params['b_params'] = {
            'topK': trial.suggest_int('b_p3_topK', 5, 2000, log=True),
            'implicit': True, 
            'normalize_similarity': True
        }
    elif params['base_b_algorithm'] == 'p3alpha':
        params['b_params'] = {
            'alpha': trial.suggest_float('b_p3a_alpha', 1e-5, 100, log=True),
            'topK': trial.suggest_int('b_p3a_topK', 5, 2000, log=True),
            'implicit': True, 
            'normalize_similarity': True
        }
    elif params['base_b_algorithm'] == 'rp3beta':
        params['b_params'] = {
            'alpha': trial.suggest_float('b_rp3_alpha', 1e-5, 100, log=True),
            'beta': trial.suggest_float('b_rp3_beta', 1e-5, 100, log=True),
            'topK': trial.suggest_int('b_rp3_topK', 5, 2000, log=True),
            'implicit': True, 
            'normalize_similarity': True
        }
    else:
        params['b_params'] = {
            'hybrid_lambda': trial.suggest_float('b_hhp_hybrid_lambda', 0.01, 0.99),
            'topK': trial.suggest_int('b_hhp_topK', 5, 2000, log=True),
        }

    recommender_instance = TwoWayRankAggregationRecommender(URM_train)
    recommender_instance.fit(**params) 
        
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10, "MAP"]

In [17]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='tuning',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_function, n_trials=200)

[I 2023-12-24 17:26:26,660] A new study created in RDB with name: tuning


TwoWayRankAggregationRecommender: URM Detected 627 ( 4.8%) users with no interactions.
TwoWayRankAggregationRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) users with no interactions.
P3alphaRecommender: URM Detected 627 ( 4.8%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 1780.28 column/sec. Elapsed time 12.55 sec
P3alphaRecommender: Similarity column 13024 (100.0%), 3858.91 column/sec. Elapsed time 3.38 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 41.82 sec. Users per second: 250


[I 2023-12-24 17:27:29,309] Trial 0 finished with value: 0.04044604146172684 and parameters: {'base_f_algorithm': 'p3alpha', 'base_b_algorithm': 'p3alpha', 'convex_lambda': 0.1637295332308224, 'f_p3a_alpha': 0.9901301652504116, 'f_p3a_topK': 143, 'b_p3a_alpha': 0.0005967050287171578, 'b_p3a_topK': 15}. Best is trial 0 with value: 0.04044604146172684.


TwoWayRankAggregationRecommender: URM Detected 627 ( 4.8%) users with no interactions.
TwoWayRankAggregationRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) users with no interactions.
P3alphaRecommender: URM Detected 627 ( 4.8%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 1739.05 column/sec. Elapsed time 12.85 sec
P3alphaRecommender: Similarity column 13024 (100.0%), 437.97 column/sec. Elapsed time 29.74 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 1.73 min. Users per second: 100


[I 2023-12-24 17:30:21,530] Trial 1 finished with value: 0.04509172222829511 and parameters: {'base_f_algorithm': 'rp3beta', 'base_b_algorithm': 'p3', 'convex_lambda': 0.14957521692815676, 'f_rp3_alpha': 9.68369085372405e-05, 'f_rp3_beta': 0.0007991828703043603, 'f_rp3_topK': 112, 'b_p3_topK': 1439}. Best is trial 1 with value: 0.04509172222829511.


...

[I 2023-12-24 21:40:00,055] Trial 197 finished with value: 0.0438476768863635 and parameters: {'base_f_algorithm': 'p3alpha', 'base_b_algorithm': 'rp3beta', 'convex_lambda': 0.5048444007254986, 'f_p3a_alpha': 0.8344857911634194, 'f_p3a_topK': 34, 'b_rp3_alpha': 4.4398101789746375e-05, 'b_rp3_beta': 0.00010484621841866483, 'b_rp3_topK': 97}. Best is trial 175 with value: 0.047955552215786255.


TwoWayRankAggregationRecommender: URM Detected 627 ( 4.8%) users with no interactions.
TwoWayRankAggregationRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) users with no interactions.
RP3betaRecommender: URM Detected 627 ( 4.8%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 3504.94 column/sec. Elapsed time 6.38 sec
RP3betaRecommender: Similarity column 13024 (100.0%), 2548.14 column/sec. Elapsed time 5.11 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 50.23 sec. Users per second: 208


[I 2023-12-24 21:41:04,156] Trial 198 finished with value: 0.04777443421260425 and parameters: {'base_f_algorithm': 'p3alpha', 'base_b_algorithm': 'rp3beta', 'convex_lambda': 0.18676371478724568, 'f_p3a_alpha': 0.3174452864649511, 'f_p3a_topK': 26, 'b_rp3_alpha': 1.3641596596280279e-05, 'b_rp3_beta': 0.0004612977023763224, 'b_rp3_topK': 52}. Best is trial 175 with value: 0.047955552215786255.


TwoWayRankAggregationRecommender: URM Detected 627 ( 4.8%) users with no interactions.
TwoWayRankAggregationRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) users with no interactions.
RP3betaRecommender: URM Detected 627 ( 4.8%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 3275.11 column/sec. Elapsed time 6.82 sec
RP3betaRecommender: Similarity column 13024 (100.0%), 2301.39 column/sec. Elapsed time 5.66 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 51.37 sec. Users per second: 204


[I 2023-12-24 21:42:10,917] Trial 199 finished with value: 0.04748083047934876 and parameters: {'base_f_algorithm': 'p3alpha', 'base_b_algorithm': 'rp3beta', 'convex_lambda': 0.26253419050222054, 'f_p3a_alpha': 0.11249995967876879, 'f_p3a_topK': 38, 'b_rp3_alpha': 0.0005356720889595254, 'b_rp3_beta': 0.00022802608800654843, 'b_rp3_topK': 75}. Best is trial 175 with value: 0.047955552215786255.


In [18]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [19]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [20]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [21]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_twra.json', 'w') as params_file:
        params = study.best_params
        json.dump(params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_twra.json', 
            'tuning_results/best_params_twra.json', 
            'TWRA tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/best_params_twra.json',
        '/kaggle/working/best_params_twra.json'
    )

In [22]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_twra.db',
        'TWRA tuning db updated results (from kaggle notebook)'
    )